In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.chdir('/Users/xiaojiezhou/Documents/DataScience/Kaggle/M5')
os.getcwd()


# Any results you write to the current directory are saved as output.

'/Users/xiaojiezhou/Documents/DataScience/Kaggle/M5'

	1. Calculate weight for the level 12 series
	2. Use the naive logic to make forecasts for each of the level 12 series
	3. Infer forecast, ground truth values, and weights for all the higher level series by aggregating
	4. Calculalte RMSSE for all series using the equation
	5. Multiply weight by respective RMSSE and add all these products

## 0. Import libraries and read in data

In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm

In [4]:
df = pd.read_feather('rawdata/sales_train_validation.feather')
price_df = pd.read_feather('rawdata/sell_prices.feather')
#submission_file = pd.read_feather('rawdata/sample_submission.feather')
cal_df = pd.read_feather('rawdata/calendar.feather')



In [6]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"

## 1. Calculate weight for the level 12 series

In [7]:
for day in tqdm(range(1858, 1886)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:20<00:00,  1.44it/s]


In [8]:
df["dollar_sales"] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1)

In [9]:
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True)

In [10]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [11]:
df.drop(columns=["dollar_sales"], inplace=True)

In [12]:
df["weight"] /= 12

## 2. Infer round truth values, and weights for all the higher level series by aggregating

In [13]:
agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0]].sum()).transpose()
id_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
for col in id_cols:
    agg_df[col] = 'all'
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [14]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [15]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
    temp_df["level"] = level
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = 'all'
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████| 10/10 [00:14<00:00,  1.84s/it]


In [16]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])

30490 12350 42840


In [17]:
agg_df["weight"].sum() + df["weight"].sum()

1.0000000000000007

In [18]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,8.333333e-02
0,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,17095,21834,23187,all,all,all,all,CA,2,3.614587e-02
1,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,10615,12266,12282,all,all,all,all,TX,2,2.328074e-02
2,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,12807,14862,14326,all,all,all,all,WI,2,2.390672e-02
0,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,4387,5577,6113,all,all,all,CA_1,all,3,9.091642e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,HOUSEHOLD_2_515,all,all,all,TX,11,2.239415e-07
9143,0,0,0,0,0,0,0,0,0,0,...,1,1,0,HOUSEHOLD_2_515,all,all,all,WI,11,4.478830e-07
9144,0,2,0,1,1,1,0,2,1,2,...,1,0,1,HOUSEHOLD_2_516,all,all,all,CA,11,2.295798e-06
9145,2,1,0,0,0,0,1,1,0,0,...,0,0,0,HOUSEHOLD_2_516,all,all,all,TX,11,2.430845e-06


## 3. Top down forecasts from different levels

In [19]:
df["last28_mean"] = df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1858, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()[0]

In [20]:
for l in tqdm(range(1, 10)):
    # 1. make forecast for this level
    this_level_df = agg_df[agg_df["level"]==l].copy()
    for d in range(1, 29):
        this_level_df["F_" + str(l) + "_" + str(1885 + d)] = this_level_df["d_" + str(1885 + d - 28)]
    
    # from this level, distribute forecast to all level 12 series
    # find columns with non 'all' entries
    important_column_ids = list(this_level_df[id_cols].columns[this_level_df[id_cols].nunique()!=1])
    this_level_df.reset_index(drop=True, inplace=True)
    for i, row in this_level_df.iterrows():
        if len(important_column_ids) == 0:
            level_mean_with_cond = this_level_df[[c for c in df.columns if c.find("d_")==0 and\
                                   int(c.split("_")[1]) in range(1858, 1886)]].transpose().mean()[0]
            proportion = df["last28_mean"] / level_mean_with_cond 
            for d in range(1, 29):
                df["F_" + str(l) + "_" + str(1885 + d)] = list(this_level_df["F_" + str(l) + "_" + str(1885 + d)])[0] * proportion
        else:
            cond = True
            for col in important_column_ids:
                cond = cond & (df[col] == row[col])
                
            level_mean_with_cond = this_level_df[[c for c in df.columns if c.find("d_")==0 and\
                                   int(c.split("_")[1]) in range(1858, 1886)]].transpose().mean()[i]
            proportion = df["last28_mean"] / level_mean_with_cond 
            for d in range(1, 29):
                df.loc[cond, "F_" + str(l) + "_" + str(1885 + d)] = row["F_" + str(l) + "_" + str(1885 + d)] * proportion
    
# remake agg_df
new_agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
id_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
for col in id_cols:
    new_agg_df[col] = 'all'
new_agg_df["level"] = 1
new_agg_df["weight"] = 1/12
column_order = new_agg_df.columns

for level in level_groupings:
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
    temp_df["level"] = level
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = 'all'
    new_agg_df = new_agg_df.append(temp_df[column_order])
del temp_df

100%|██████████| 9/9 [00:18<00:00,  3.29s/it]


In [21]:
agg_df = new_agg_df

## 4. Calculalte RMSSE for all series using the equation

In [22]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [23]:
train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]

forecast_cols_dict = {}
for i in range(1, 10):
    forecast_cols_dict[i] = [c for c in df.columns if c.find("F_"+str(i)+"_") == 0]

In [24]:
for i in range(1, 10):
    df["rmsse_" + str(i)] = rmsse(np.array(df[ground_truth_cols]), 
        np.array(df[forecast_cols_dict[i]]), np.array(df[train_series_cols]))
    agg_df["rmsse_" + str(i)] = rmsse(np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[forecast_cols_dict[i]]), np.array(agg_df[train_series_cols]))

In [25]:
for i in range(1, 10):
    df["wrmsse_" + str(i)] = df["weight"] * df["rmsse_" + str(i)]
    agg_df["wrmsse_" + str(i)] = agg_df["weight"] * agg_df["rmsse_" + str(i)]

In [26]:
for i in range(1, 10):
    print("Aggregation by level", str(i) + ":")
    print(df["wrmsse_" + str(i)].sum() + agg_df["wrmsse_" + str(i)].sum())
    print()

Aggregation by level 1:
0.7526969415454309

Aggregation by level 2:
0.7583278269835372

Aggregation by level 3:
0.7640636603285994

Aggregation by level 4:
0.7573295317439488

Aggregation by level 5:
0.7543085508691126

Aggregation by level 6:
0.7635048362696769

Aggregation by level 7:
0.761857669668642

Aggregation by level 8:
0.7717987726658402

Aggregation by level 9:
0.772688952163536



###### Make submission file

In [27]:
df["last28_mean"] = df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1886, 1914)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()[0]

In [28]:
level_1_mean = agg_df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1886, 1914)]].transpose().mean().reset_index()[0][0]

In [29]:
submit_df = df[["id"]]
for i in range(1, 29):
    proportion = df["last28_mean"] / level_1_mean
    submit_df["F" + str(i)] = agg_df[agg_df["level"]==1]["d_" + str(1885 + i)][0] * proportion

/Users/xiaojiezhou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
submit_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.822141,0.772321,0.738150,0.791117,1.038734,1.201515,1.287125,0.921985,0.905219,...,0.953260,1.103198,1.177973,0.868174,0.857019,0.806218,0.799147,0.924244,1.116885,1.135887
1,HOBBIES_1_002_CA_1_validation,0.060899,0.057209,0.054678,0.058601,0.076943,0.089001,0.095343,0.068295,0.067053,...,0.070612,0.081718,0.087257,0.064309,0.063483,0.059720,0.059196,0.068463,0.082732,0.084140
2,HOBBIES_1_003_CA_1_validation,0.487194,0.457672,0.437422,0.468810,0.615546,0.712009,0.762741,0.546362,0.536426,...,0.564895,0.653747,0.698058,0.514473,0.507863,0.477759,0.473569,0.547700,0.661858,0.673118
3,HOBBIES_1_004_CA_1_validation,1.552932,1.458828,1.394282,1.494333,1.962052,2.269528,2.431237,1.741528,1.709859,...,1.800602,2.083819,2.225061,1.639884,1.618814,1.522857,1.509500,1.745794,2.109671,2.145564
4,HOBBIES_1_005_CA_1_validation,1.157087,1.086970,1.038877,1.113424,1.461921,1.691021,1.811510,1.297609,1.274012,...,1.341625,1.552649,1.657889,1.221874,1.206175,1.134678,1.124725,1.300788,1.571912,1.598655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.182698,0.171627,0.164033,0.175804,0.230830,0.267003,0.286028,0.204886,0.201160,...,0.211835,0.245155,0.261772,0.192927,0.190449,0.179160,0.177588,0.205388,0.248197,0.252419
30486,FOODS_3_824_WI_3_validation,0.274047,0.257440,0.246050,0.263706,0.346245,0.400505,0.429042,0.307328,0.301740,...,0.317753,0.367733,0.392658,0.289391,0.285673,0.268739,0.266382,0.308081,0.372295,0.378629
30487,FOODS_3_825_WI_3_validation,0.761241,0.715112,0.683472,0.732516,0.961790,1.112514,1.191783,0.853690,0.838166,...,0.882648,1.021480,1.090716,0.803865,0.793536,0.746498,0.739951,0.855781,1.034153,1.051747
30488,FOODS_3_826_WI_3_validation,0.791691,0.743716,0.710811,0.761817,1.000262,1.157014,1.239454,0.887838,0.871693,...,0.917954,1.062339,1.134345,0.836019,0.825278,0.776358,0.769549,0.890013,1.075519,1.093817


In [31]:
submit_df2 = submit_df.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [32]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)

In [33]:
submit_df.to_csv("resuls/submission_topdown.csv", index=False)